# Breaking Out of the Loop
## Refactoring Legacy Software with Polars

In this quick tutorial, we'll be creating a very limited version of the Global Summary of the Month. By the end, you should have a handy cookbook for whipping up solutions to common data-crunching requirements.


### Part 1 - Reading Your Input
***

Let's start by importing Polars. We'll also import a library called itables to make exploring our tables easier in the notebook.

In [1]:
import polars as pl
#import itables
#itables.init_notebook_mode(connected=False)

For today's example, we'll be using real NOAA data. More specifically, we'll be pulling station files from the Global Historical Climate Network's daily dataset. These are just plain text "flat files" with fixed column widths. Daily values are limited to 5 spaces and are followed by measurement, QC, and source flags--each taking up a single space:

<img src="imgs/dly.png" alt="image" width="500" height="auto">

Unfortunately, Polars doesn't have a native method for reading fixed-width files, so I'm including a utility class here. You'll find it in the GitHub repo as **InputOutputUtils.py.**

In [2]:
import InputOutputUtils as io
from importlib import reload
reload(io) #Prevents caching in Python modules
#JFK International Airport
station_file = "stations/USW00094789.dly"
df = io.dlyAsDataFrame( station_file )
df.head()
#show(df, layout={"topStart": None, "topEnd": None})

STATION,DATE,Element,daily_values,qc_flags,days_in_month
str,date,str,list[f32],list[str],i32
"""USW00094789""",1948-07-01,"""TMAX""","[-9999.0, -9999.0, … 317.0]","["""", """", … """"]",31
"""USW00094789""",1948-07-01,"""TMIN""","[-9999.0, -9999.0, … 233.0]","["""", """", … """"]",31
"""USW00094789""",1948-07-01,"""PRCP""","[-9999.0, -9999.0, … 0.0]","["""", """", … """"]",31
"""USW00094789""",1948-07-01,"""SNOW""","[-9999.0, -9999.0, … 0.0]","["""", """", … """"]",31
"""USW00094789""",1948-07-01,"""SNWD""","[-9999.0, -9999.0, … 0.0]","["""", """", … """"]",31




What's happening inside of input_output_utils is outside the scope of this talk, but it's a useful class. Shoutout to (GET NAME) over on StackOverflow for posting the snippet I used to create it. The important thing is, we've converted our fixed-width flat file into a Polars Dataframe.

### Part 2 - Data Crunching
***

The real GSOM puts out more than 63 possible element-columns. In this quick demonstration, our program will only put out 5 elements:
* Average Minimum Temperature
* Average Maximum Temperature
* Average Temperature
* Cooling Degree Days
* Heating Degree Days

#### 2A - Filtering
***

To create these elements, we'll only need to keep rows where Element is equal to TMIN and TMAX. Let's go ahead and filter those elements:


In [3]:
#Create an array with the elements we want to keep
needed = ["TMIN", "TMAX"]

#only show rows where "Element" matches strings in this list
df = df.filter(
    pl.col("Element").is_in(needed)
)

df.select(["STATION", "Element", "DATE", "daily_values", "qc_flags"]).head()

STATION,Element,DATE,daily_values,qc_flags
str,str,date,list[f32],list[str]
"""USW00094789""","""TMAX""",1948-07-01,"[-9999.0, -9999.0, … 317.0]","["""", """", … """"]"
"""USW00094789""","""TMIN""",1948-07-01,"[-9999.0, -9999.0, … 233.0]","["""", """", … """"]"
"""USW00094789""","""TMAX""",1948-08-01,"[233.0, 261.0, … 283.0]","["""", """", … """"]"
"""USW00094789""","""TMIN""",1948-08-01,"[211.0, 200.0, … 172.0]","["""", """", … """"]"
"""USW00094789""","""TMAX""",1948-09-01,"[256.0, 250.0, … -9999.0]","["""", """", … """"]"


If you look at the shape of the dataframes, you'll see our row count has dropped from 15, 584 rows to 1,842. You don't necessarily need to drive yourself nuts trying to whittle down the size of your dataset, but if it's 700% larger than it needs to be, your speed will suffer.


#### 2B - Data Validation
***

Before we run our calculations, let's first delete any invalid data in our daily_value lists. In the real GSOM, we'd remove any daily values where the corresponding QC flag wasn't an empty string. For the purposes of this demo, we're only going to remove the values that are equal to -9999.0.

In [4]:

df = df.with_columns(
        #We could have simply replaced daily_values, but we're not done with it yet!
        filtered_values = pl.col("daily_values").list.eval(
            pl.element().filter(pl.element() != -9999.0)
        )
    )

df.select(
    ["DATE", "Element", "daily_values", "filtered_values"]
    ).head()

DATE,Element,daily_values,filtered_values
date,str,list[f32],list[f32]
1948-07-01,"""TMAX""","[-9999.0, -9999.0, … 317.0]","[233.0, 272.0, … 317.0]"
1948-07-01,"""TMIN""","[-9999.0, -9999.0, … 233.0]","[178.0, 211.0, … 233.0]"
1948-08-01,"""TMAX""","[233.0, 261.0, … 283.0]","[233.0, 261.0, … 283.0]"
1948-08-01,"""TMIN""","[211.0, 200.0, … 172.0]","[211.0, 200.0, … 172.0]"
1948-09-01,"""TMAX""","[256.0, 250.0, … -9999.0]","[256.0, 250.0, … 239.0]"


Once we've filtered our daily_values, we can subtract it's length from the days_in_month column. We'll then filter out any rows where we have 4 or more missing_days. 

In [5]:
df = df.with_columns(
        missing_days = pl.col("days_in_month") - pl.col("filtered_values").list.len()
    ).filter(
        pl.col("missing_days") < 4
    )

df.select(
        ["DATE", "Element", "daily_values", "filtered_values", "missing_days"]
    ).head()

DATE,Element,daily_values,filtered_values,missing_days
date,str,list[f32],list[f32],i64
1948-08-01,"""TMAX""","[233.0, 261.0, … 283.0]","[233.0, 261.0, … 283.0]",0
1948-08-01,"""TMIN""","[211.0, 200.0, … 172.0]","[211.0, 200.0, … 172.0]",0
1948-09-01,"""TMAX""","[256.0, 250.0, … -9999.0]","[256.0, 250.0, … 239.0]",0
1948-09-01,"""TMIN""","[144.0, 150.0, … -9999.0]","[144.0, 150.0, … 167.0]",0
1948-10-01,"""TMAX""","[256.0, 261.0, … 200.0]","[256.0, 261.0, … 200.0]",0


#### 2C - Easy Requirements
***
Now that we have a cleaned up version of our daily_values, we can create a monthly summary for Average Minimum Temperature and Average Maximum Temperature. We'll also need to scale the value down by 90%. Luckily, we can handle this all in a single line!

In [6]:
#TODO Run .mean() on TMIN and TMAX
df = df.with_columns(
    value = pl.col("filtered_values").list.mean() * .1
    
)

df.select(["DATE", "Element", "filtered_values", "value"]).head()

DATE,Element,filtered_values,value
date,str,list[f32],f32
1948-08-01,"""TMAX""","[233.0, 261.0, … 283.0]",27.812902
1948-08-01,"""TMIN""","[211.0, 200.0, … 172.0]",19.493549
1948-09-01,"""TMAX""","[256.0, 250.0, … 239.0]",24.969999
1948-09-01,"""TMIN""","[144.0, 150.0, … 167.0]",15.313334
1948-10-01,"""TMAX""","[256.0, 261.0, … 200.0]",17.670967


#### 2C - Challenging Requirements
***
We've successfully run calculations on our base elements and decided which values we need to discard using only native Polars expressions. What if we need to compare two base elements? **FOR EXAMPLE**, GSOM also produces Average Temperature by comparing Average Max Temp and Average Min Temp. The real GSOM actually just adds TMAX & TMIN and then divides by two. What if we wanted to be a little more precise and create a list of daily averages and then mean() that? How am I supposed to make it human readable? How does that work?

We're going to give ourselves permission to add as many columns as we need to. Just drop them when you're done with your calculations. "It's free real estate." Polars and Panda are meant to operate in columnwise fashion. Operating over many columns is absolutely no problem.

The first step will be creating both "sides" of our new TAVG elements.

In [7]:
#TODO Create TAVG with the "free real estate" method
#Let's start by grabbing all of the TMIN. They'll be the "left side" of our join.
tavg = df.filter(pl.col("Element")=="TMIN")

#Now let's grab our tmax rows and change the name of our list columns
tmax = df.filter(
    pl.col("Element")=="TMAX"
).with_columns(
    daily_values_2 = pl.col("daily_values"),
    qc_flags_2 = pl.col("qc_flags")
)#.select(["DATE", "Element", "daily_values_2", "qc_flags_2"])

#We change the element name to prepare for the join. 
#We use pl.lit() for literal values
tmax = tmax.with_columns(
    Element = pl.lit("TMIN")
)

tmax.head()

STATION,DATE,Element,daily_values,qc_flags,days_in_month,filtered_values,missing_days,value,daily_values_2,qc_flags_2
str,date,str,list[f32],list[str],i32,list[f32],i64,f32,list[f32],list[str]
"""USW00094789""",1948-08-01,"""TMIN""","[233.0, 261.0, … 283.0]","["""", """", … """"]",31,"[233.0, 261.0, … 283.0]",0,27.812902,"[233.0, 261.0, … 283.0]","["""", """", … """"]"
"""USW00094789""",1948-09-01,"""TMIN""","[256.0, 250.0, … -9999.0]","["""", """", … """"]",30,"[256.0, 250.0, … 239.0]",0,24.969999,"[256.0, 250.0, … -9999.0]","["""", """", … """"]"
"""USW00094789""",1948-10-01,"""TMIN""","[256.0, 261.0, … 200.0]","["""", """", … """"]",31,"[256.0, 261.0, … 200.0]",0,17.670967,"[256.0, 261.0, … 200.0]","["""", """", … """"]"
"""USW00094789""",1948-11-01,"""TMIN""","[172.0, 128.0, … -9999.0]","["""", """", … """"]",30,"[172.0, 128.0, … 72.0]",0,14.173334,"[172.0, 128.0, … -9999.0]","["""", """", … """"]"
"""USW00094789""",1948-12-01,"""TMIN""","[106.0, 83.0, … 100.0]","["""", """", … """"]",31,"[106.0, 83.0, … 100.0]",0,6.551613,"[106.0, 83.0, … 100.0]","["""", """", … """"]"


Let's use the join() expression to create our TAVG rows with everything they'll need to be calculated.

In [8]:
tavg = tavg.join(
            tmax, on=["DATE", "Element"], how="left"
        ).with_columns(
            Element = pl.lit("TAVG")
        )
#Saving the columns we need for the vertical stack manually
#In this case I've only got 11 columns, so brute force is ok
#If you try to vstack tables with mismatched width, you're gonna have a real bad time.
tavg = tavg.select(["STATION", "DATE", "Element", "daily_values", "qc_flags", "days_in_month", "filtered_values", "missing_days", "value", "daily_values_2", "qc_flags_2"])
tavg.head()

STATION,DATE,Element,daily_values,qc_flags,days_in_month,filtered_values,missing_days,value,daily_values_2,qc_flags_2
str,date,str,list[f32],list[str],i32,list[f32],i64,f32,list[f32],list[str]
"""USW00094789""",1948-08-01,"""TAVG""","[211.0, 200.0, … 172.0]","["""", """", … """"]",31,"[211.0, 200.0, … 172.0]",0,19.493549,"[233.0, 261.0, … 283.0]","["""", """", … """"]"
"""USW00094789""",1948-09-01,"""TAVG""","[144.0, 150.0, … -9999.0]","["""", """", … """"]",30,"[144.0, 150.0, … 167.0]",0,15.313334,"[256.0, 250.0, … -9999.0]","["""", """", … """"]"
"""USW00094789""",1948-10-01,"""TAVG""","[167.0, 161.0, … 67.0]","["""", """", … """"]",31,"[167.0, 161.0, … 67.0]",0,8.841936,"[256.0, 261.0, … 200.0]","["""", """", … """"]"
"""USW00094789""",1948-11-01,"""TAVG""","[100.0, 56.0, … -9999.0]","["""", """", … """"]",30,"[100.0, 56.0, … -6.0]",0,6.6,"[172.0, 128.0, … -9999.0]","["""", """", … """"]"
"""USW00094789""",1948-12-01,"""TAVG""","[22.0, -6.0, … -39.0]","["""", """", … """"]",31,"[22.0, -6.0, … -39.0]",0,-1.148387,"[106.0, 83.0, … 100.0]","["""", """", … """"]"


In [9]:
#Add blank columns for daily_values_2 and qc_flags_2 
#so we don't break the vstack
df = df.with_columns(
    daily_values_2 = [],
    qc_flags_2 = []
).with_columns(
    pl.col("daily_values_2").cast(pl.List(pl.Float32)),
    pl.col("qc_flags_2").cast(pl.List(pl.String))
)

#Finally insert our derived 
df = df.vstack(tavg).sort("DATE")
df.head()

STATION,DATE,Element,daily_values,qc_flags,days_in_month,filtered_values,missing_days,value,daily_values_2,qc_flags_2
str,date,str,list[f32],list[str],i32,list[f32],i64,f32,list[f32],list[str]
"""USW00094789""",1948-08-01,"""TMAX""","[233.0, 261.0, … 283.0]","["""", """", … """"]",31,"[233.0, 261.0, … 283.0]",0,27.812902,[],[]
"""USW00094789""",1948-08-01,"""TMIN""","[211.0, 200.0, … 172.0]","["""", """", … """"]",31,"[211.0, 200.0, … 172.0]",0,19.493549,[],[]
"""USW00094789""",1948-08-01,"""TAVG""","[211.0, 200.0, … 172.0]","["""", """", … """"]",31,"[211.0, 200.0, … 172.0]",0,19.493549,"[233.0, 261.0, … 283.0]","["""", """", … """"]"
"""USW00094789""",1948-09-01,"""TMAX""","[256.0, 250.0, … -9999.0]","["""", """", … """"]",30,"[256.0, 250.0, … 239.0]",0,24.969999,[],[]
"""USW00094789""",1948-09-01,"""TMIN""","[144.0, 150.0, … -9999.0]","["""", """", … """"]",30,"[144.0, 150.0, … 167.0]",0,15.313334,[],[]


In [10]:
tmin_vals = ["tmin_" + str(i) for i in range(31)]
tmax_vals = ["tmax_" + str(i) for i in range(31)]
min_qc_flags = ["min_qc_" + str(i) for i in range(31)]
max_qc_flags = ["max_qc_" + str(i) for i in range(31)]

df = df.with_columns(
        df.select( 
            pl.col.daily_values.list.to_struct(fields=tmin_vals)
        ).unnest("daily_values")
    ).with_columns(
         df.select( 
            pl.col.daily_values_2.list.to_struct(fields=tmax_vals)
        ).unnest("daily_values_2")
    ).with_columns(
        df.select( 
            pl.col.qc_flags.list.to_struct(fields=min_qc_flags)
        ).unnest("qc_flags")
    ).with_columns(
        df.select( 
            pl.col.qc_flags_2.list.to_struct(fields=max_qc_flags)
        ).unnest("qc_flags_2")
    )

df.head()

STATION,DATE,Element,daily_values,qc_flags,days_in_month,filtered_values,missing_days,value,daily_values_2,qc_flags_2,tmin_0,tmin_1,tmin_2,tmin_3,tmin_4,tmin_5,tmin_6,tmin_7,tmin_8,tmin_9,tmin_10,tmin_11,tmin_12,tmin_13,tmin_14,tmin_15,tmin_16,tmin_17,tmin_18,tmin_19,tmin_20,tmin_21,tmin_22,tmin_23,tmin_24,tmin_25,…,min_qc_25,min_qc_26,min_qc_27,min_qc_28,min_qc_29,min_qc_30,max_qc_0,max_qc_1,max_qc_2,max_qc_3,max_qc_4,max_qc_5,max_qc_6,max_qc_7,max_qc_8,max_qc_9,max_qc_10,max_qc_11,max_qc_12,max_qc_13,max_qc_14,max_qc_15,max_qc_16,max_qc_17,max_qc_18,max_qc_19,max_qc_20,max_qc_21,max_qc_22,max_qc_23,max_qc_24,max_qc_25,max_qc_26,max_qc_27,max_qc_28,max_qc_29,max_qc_30
str,date,str,list[f32],list[str],i32,list[f32],i64,f32,list[f32],list[str],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""USW00094789""",1948-08-01,"""TMAX""","[233.0, 261.0, … 283.0]","["""", """", … """"]",31,"[233.0, 261.0, … 283.0]",0,27.812902,[],[],233.0,261.0,239.0,239.0,222.0,233.0,256.0,250.0,278.0,267.0,261.0,239.0,272.0,289.0,311.0,289.0,267.0,244.0,261.0,222.0,267.0,256.0,272.0,261.0,344.0,378.0,…,"""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""USW00094789""",1948-08-01,"""TMIN""","[211.0, 200.0, … 172.0]","["""", """", … """"]",31,"[211.0, 200.0, … 172.0]",0,19.493549,[],[],211.0,200.0,189.0,206.0,194.0,167.0,150.0,161.0,172.0,167.0,161.0,206.0,206.0,189.0,172.0,189.0,178.0,183.0,200.0,194.0,183.0,183.0,183.0,194.0,217.0,244.0,…,"""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""USW00094789""",1948-08-01,"""TAVG""","[211.0, 200.0, … 172.0]","["""", """", … """"]",31,"[211.0, 200.0, … 172.0]",0,19.493549,"[233.0, 261.0, … 283.0]","["""", """", … """"]",211.0,200.0,189.0,206.0,194.0,167.0,150.0,161.0,172.0,167.0,161.0,206.0,206.0,189.0,172.0,189.0,178.0,183.0,200.0,194.0,183.0,183.0,183.0,194.0,217.0,244.0,…,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""USW00094789""",1948-09-01,"""TMAX""","[256.0, 250.0, … -9999.0]","["""", """", … """"]",30,"[256.0, 250.0, … 239.0]",0,24.969999,[],[],256.0,250.0,244.0,256.0,261.0,256.0,261.0,261.0,278.0,233.0,250.0,272.0,278.0,272.0,233.0,217.0,217.0,300.0,278.0,267.0,233.0,217.0,211.0,189.0,217.0,228.0,…,"""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""USW00094789""",1948-09-01,"""TMIN""","[144.0, 150.0, … -9999.0]","["""", """", … """"]",30,"[144.0, 150.0, … 167.0]",0,15.313334,[],[],144.0,150.0,183.0,172.0,183.0,167.0,178.0,189.0,211.0,172.0,161.0,178.0,200.0,172.0,133.0,117.0,106.0,178.0,189.0,194.0,139.0,100.0,100.0,128.0,122.0,100.0,…,"""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


**135 columns?** Have you lost your mind, Brodie?!

Just remember your mantra, kids:

<img src="imgs/free_real_estate.png" alt="image" width="500" height="auto">


OK. I promise we're almost there! So how are we supposed to write a query that compares 125 columns? We're going to dynamically build 31 queries using a loop. I know I titled the presentation "Breaking Out of the Loop", but we're really going to want to make an exception here. 

In [21]:
daily_average_expressions = []
daily_averages = ["da_" + str(i) for i in range(31)]

for v, v2, qc, qc2, dv in zip(tmin_vals, tmax_vals, min_qc_flags, max_qc_flags, daily_averages):
    daily_average_expressions.append(
        pl.when(
            pl.col("Element").is_in(["TAVG"]) 
            & ~pl.col( v ).is_in([-9999.0])
            & pl.col( qc ).is_in([""])
            & ~pl.col( v2 ).is_in([-9999.0])
            & pl.col( qc2 ).is_in([""])
        ).then(
            ((pl.col(v) + pl.col(v2))/2)*.1
        ).otherwise(
            -9999.0
        ).alias(dv)
    )

#
import polars.selectors as cs

df = df.with_columns(
            daily_average_expressions
        ).with_columns(
            daily_averages = pl.concat_list(cs.matches(".*da_.*"))
        )

df.filter(pl.col.Element.is_in(["TAVG"])).select(["Element", "DATE", "daily_averages"]).head()

Element,DATE,daily_averages
str,date,list[f32]
"""TAVG""",1948-08-01,"[22.200001, 23.050001, … 22.75]"
"""TAVG""",1948-09-01,"[20.0, 20.0, … -9999.0]"
"""TAVG""",1948-10-01,"[21.15, 21.1, … 13.35]"
"""TAVG""",1948-11-01,"[13.6, 9.2, … -9999.0]"
"""TAVG""",1948-12-01,"[6.4, 3.85, … 3.05]"


#### 2D - "Tap Out" Requirements
***
Ok, so you really tried your best to use native Polars or Pandas expressions, but neither you nor the AI can get the right output. What now? Now we're going to create a custom, "Just in time" compiled function using Numba.

In [ ]:
#TODO Create CLDD & HTDD with a guvectorize function

### Part 3 - Creating the Output CSV
***

In [ ]:
#TODO Pivot the dataframe to the final structure

In [ ]:
#TODO Write to CSV to local